## Dataframe making

In [1]:
from twitterscraper import query_tweets
import codecs, json
import re
import operator
import copy
import nltk
import numpy as np
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, SpaceTokenizer, PunktSentenceTokenizer
from collections import Counter
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

INFO: {'User-Agent': 'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16'}


In [2]:
with codecs.open('collection.json', 'r', 'utf-8') as f:
    tw = json.load(f, encoding = 'utf-8')

In [3]:
tweet_table = pd.DataFrame(tw)

In [4]:
tweet_table

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,screen_name,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url
0,True,"[cambridgemask, forestfires, australiafires, p...",[https://pbs.twimg.com/media/D06CH-jXQAAnY4Z.jpg],False,False,1,[https://buff.ly/2EO6uxC],,0,[],...,CambridgeMaskCo,'Act immediately to survive': Fires rip to Aus...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2019-03-05T16:00:15,1551801615,1102962033849438209,/CambridgeMaskCo/status/1102962033849438209,3297612197,Cambridge Mask Co,
1,False,"[AustraliaHeat, AustraliaFires, Worldnews]",[],False,False,0,[https://reut.rs/2SEDPi8],,0,[],...,DButler1958,Record Australian heat brings fire to a scorch...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2019-03-03T14:45:25,1551624325,1102218424145580038,/DButler1958/status/1102218424145580038,920326607558135809,Donna Butler,
2,False,"[GlobalWaming, climatechange, Australiafires, ...",[],False,False,1,[],,0,[],...,gholmbo,"If you're looking for global warming, go to t...","<p class=""TweetTextSize js-tweet-text tweet-te...",2019-02-07T20:06:52,1549570012,1093602011470213120,/gholmbo/status/1093602011470213120,35784771,Gary Holmbo,
3,False,"[AUSTRALIAFIRES, covid19australia, auspol2020]",[],False,True,0,[],1248780341440999427,0,"[{'screen_name': 'no_fkng_way', 'user_id': '93...",...,gayzieh,You’re kidding after the and they should hav...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-04-11T23:57:31,1586649451,1249124449762320388,/gayzieh/status/1249124449762320388,549611551,GLH,
4,True,[AUSTRALIAFIRES],[https://pbs.twimg.com/media/EVWz7oTUwAEfPhA.jpg],True,False,1,[https://youtu.be/kqRcLoLkMWI],,1,[],...,Infiniteskiesyt,Landing in on infinite fight simWatch here:,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-04-11T22:29:35,1586644175,1249102324179456003,/Infiniteskiesyt/status/1249102324179456003,1030245736187813888,Infinite Skies,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122559,False,"[bushfires, AustraliaisBurning, AustralianBush...",[],False,False,8,[https://www.broadsheet.com.au/melbourne/food-...,,0,[],...,leesungyeols,here is a compilation of restaurants/cafes/bar...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-07T08:07:18,1578384438,1214458472919531521,/leesungyeols/status/1214458472919531521,169833002,alina ☾,
122560,False,"[ClimateFires, AustralianBushfireDisaster]",[],False,False,0,[https://twitter.com/raejohnston/status/121444...,,0,[],...,sarahcalverley,I thought my heart was already broken by the d...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-07T07:20:02,1578381602,1214446581098807304,/sarahcalverley/status/1214446581098807304,281946601,Sarah Calverley,
122561,False,"[ClimateCrisis, AustralianBushfireDisaster]",[],False,False,10,[https://media.oxfam.org.au/2020/01/oxfam-sadd...,,0,[],...,OxfamAustralia,As we work together to assist and support thos...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-07T06:27:00,1578378420,1214433231858208769,/OxfamAustralia/status/1214433231858208769,13752782,Oxfam Australia,
122562,True,"[Australiabushfires, Australianbushfiredisaster]",[https://pbs.twimg.com/media/ENpx3NvXsAEIII9.png],False,False,13,[http://bit.ly/2Fs6MK0],,0,[],...,bendigobank,The national bushfire disaster appeal has now ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-07T04:47:42,1578372462,1214408245093322752,/bendigobank/status/1214408245093322752,18203593,Bendigo Bank,


In [5]:
punct_list = list(punctuation)
punct_list.append('``')
noise = stopwords.words('english') + punct_list

In [116]:
for j in ['…', '\'', '’', '', '\'s', '...', '/…', 'n\'t', '\'\'', 'us', '“', '”', '\'re', 'one', '\'m', 'australia', 'this']:
    noise.append(j)

for i in range(0, 2050):
    noise.append(str(i))

In [117]:
lemmas = WordNetLemmatizer()

In [118]:
preprop_text = []
for i in range(len(tweet_table['text'])):
    the_text = tweet_table['text'][i]
    tok_list = word_tokenize(the_text)   
    sent = ''
    for word in tok_list:
        if word.lower() not in noise:
            my_word = str(word.lower())
            my_word_num = re.sub(r" .*?[0-9]*.*? ", " ", my_word)
            my_word_num = re.sub(r"[.\'/]", "", my_word_num)
            sent += lemmas.lemmatize(my_word_num)
            sent += ' '
    preprop_text.append(sent)

In [119]:
tweet_table['preprop'] = preprop_text

In [120]:
tweet_table.sort_values(by = 'timestamp')

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,preprop
91122,False,"[AustraliaBurns, Straya, Summer, Heatwave]",[],True,False,1,[https://twitter.com/BOM_NSW/status/1085430520...,,1,[],...,Hot hot hot some temps above 48 degrees Celsiu...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2019-01-16T08:55:21,1547628921,1085460486273953793,/fionacrain/status/1085460486273953793,284508134,Fiona Crain,,hot hot hot temp degree celsius come
90273,True,"[PrayforAustralia, sedfoursquarepic]",[https://pbs.twimg.com/media/DxoLdwZW0AA9w14.jpg],False,False,0,[],,0,[],...,: Agree with us for a mighty move of God among...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2019-01-23T22:00:48,1548280848,1088194867963064321,/sedfoursquare/status/1088194867963064321,132243186,Southeast District,,agree mighty move god among pastor congregatio...
98656,True,"[AustraliaBushfires, AustraliaHeatwave, Climat...",[https://pbs.twimg.com/media/DxukergUwAA2nQq.jpg],False,False,0,[https://blog.gogreenr12.org/melbourne-heatwav...,,0,[],...,Melbourne heatwave: city expecting 44C as Vict...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2019-01-25T03:49:21,1548388161,1088644971211546625,/gogreenr12/status/1088644971211546625,2492350526,GoGreen SOCCSKSARGEN,,melbourne heatwave city expecting 44c victoria...
91121,False,[AustraliaBurns],[],False,False,0,[https://twitter.com/davisjonesn/status/109200...,,0,[],...,...thoughts with all... x …,"<p class=""TweetTextSize js-tweet-text tweet-te...",2019-02-03T11:20:11,1549192811,1092019916775993345,/Meldawson6/status/1092019916775993345,2284877388,Mel dawson,,thought x
90272,False,"[prayforaustralia, prophetofthenations]",[],False,False,0,[https://www.instagram.com/p/BtfJGVbhfV6/?utm_...,,0,[],...,Prophetic Update: Flooding in Australia …,"<p class=""TweetTextSize js-tweet-text tweet-te...",2019-02-05T04:26:06,1549340766,1092640486064414721,/MichaelHunter68/status/1092640486064414721,2914377539,Dr. Michael Hunter,,prophetic update flooding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107088,False,"[Ireland, ClimateEmergency, ClimateAction, Cli...",[],False,False,1,[https://twitter.com/Irishwildlife/status/1249...,,0,[],...,Shameful but unsurprising because do not care...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-04-12T23:46:41,1586735201,1249484113695956994,/ZazaFL/status/1249484113695956994,207950470,ZazaFL,,shameful unsurprising care environment
107087,False,"[auspol, qldpol, ClimateEmergency, GreenNewDea...",[],False,False,5,[https://reneweconomy.com.au/aemo-declares-sys...,,0,[],...,AEMO declares system strength shortfall in Que...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-04-12T23:48:57,1586735337,1249484682535636992,/Jackthelad1947/status/1249484682535636992,172081908,John Pratt,,aemo declares system strength shortfall queens...
107086,True,"[ClimateChange, ClimateCrisis, GlobalWarming, ...",[https://pbs.twimg.com/media/EVcOQWjXsAU8ZP9.jpg],False,False,2,"[https://youtu.be/Jty4XZayDlM, https://slidesh...",,0,[],...,ECOLOGICAL RESPONSIBILITY: Our Self-InterestYo...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-04-12T23:50:07,1586735407,1249484977089261570,/MassCreativity/status/1249484977089261570,239555475,Efiong Etuk,,ecological responsibility self-interestyoutube...
107085,False,"[auspol, qldpol, StopAdani, ClimateEmergency, ...",[],False,False,9,[https://reneweconomy.com.au/calls-for-morriso...,,0,[],...,Calls for Morrison to end fossil fuel stimulus...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-04-12T23:51:46,1586735506,1249485390454513667,/Jackthelad1947/status/1249485390454513667,172081908,John Pratt,,call morrison end fossil fuel stimulus redirec...


In [121]:
tweet_2020 = tweet_table.query('timestamp >= "2020-01-01T00:00:00" and timestamp <= "2020-02-20T00:00:00"')

In [122]:
tweet_2020.sort_values(by = 'timestamp')

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,preprop
67045,False,"[auspol, AustraliaOnFire]",[],False,False,0,[https://twitter.com/RDNS_TAI/status/121183481...,,0,[],...,Water - a commidity that is bought and sold to...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-01T01:15:47,1577841347,1212180587672289281,/AYi51887990/status/1212180587672289281,1176387583011516416,A Yi,,water commidity bought sold highest bidder ele...
67044,False,[australiaonfire],[],False,True,0,[],1212160929225486342,0,"[{'screen_name': 'padraigegan', 'user_id': '10...",...,A great photo. Photography is a great ho by an...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-01T02:12:19,1577844739,1212194810888515584,/Azurepink/status/1212194810888515584,74924332,Azure,,great photo photography great ho interest woul...
67043,False,"[AustraliaOnFire, AusPolhttps]",[],False,False,1,[https://twitter.com/JoeHockey/status/12119845...,,0,[],...,No the people are distressed you fool! :/…,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-01T03:49:39,1577850579,1212219308790575104,/CCC_Communities/status/1212219308790575104,854046308,Caring Communities,,people distressed fool
67042,True,[auspol],[https://pbs.twimg.com/media/ENLpTYeVUAUF9BA.j...,False,False,0,[],,0,[],...,"This is true compassion, true heartbreak, t...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-01T08:21:45,1577866905,1212287785324797952,/lunar_aphrodite/status/1212287785324797952,65484074,Charis Constantine,,true compassion true heartbreak true leadership
67041,True,"[Wildfires, AustraliaOnFire, FearGod, ObeyGod,...",[https://pbs.twimg.com/media/ENFxsTjX0AA6CTX.jpg],False,False,1,[https://cbsn.ws/37u9sD7],,0,[],...,"For the LORD thy God is a consuming fire, even...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-01T17:05:02,1577898302,1212419473900216322,/Frukta_Gud/status/1212419473900216322,921809679180881923,Frukta Gud,,lord thy god consuming fire even jealous godde...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,True,"[artists, koala, AustraliaFires, digitalart, i...",[https://pbs.twimg.com/media/ERLItxxW4AYErz6.jpg],False,False,1,[],,0,[],...,Redid this whole thing..Aren't koalas just the...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-19T23:03:26,1582153406,1230266672235282432,/MobeArt/status/1230266672235282432,1170739797952012294,mobe_art,,redid whole thingare koala best saddens happened
98999,False,[AustraliaBushfires],[],False,False,1,[],,0,[],...,"Our amazing, thoughtful Mini Vinnies have orga...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-19T23:09:59,1582153799,1230268321108578304,/stgregoryscps/status/1230268321108578304,776894294703628288,St Gregory's,,amazing thoughtful mini vinnies organised cake...
680,False,"[auspol, AustraliaFires, FireFightAustraliahttps]",[],False,False,0,[https://insidestory.org.au/time-to-think-diff...,,0,[],...,In response to your call to think differently...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-19T23:43:48,1582155828,1230276830055133184,/Jayrgoodall/status/1230276830055133184,3149027809,Jay Goodall,,response call think differently- differently v...
679,True,"[wildfires, donationsforAustralia, AustraliaFi...",[https://pbs.twimg.com/media/ERLSoyeUEAECQ3f.jpg],False,False,1,[],,0,[],...,So you think your international donation is go...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-19T23:45:29,1582155929,1230277256179666947,/SarahHa46398397/status/1230277256179666947,1228502896414560256,Sarah Hart,,think international donation going directly ne...


In [123]:
print(np.array(tweet_2020['preprop'])[12])
print(len(np.array(tweet_2020['preprop'])))

believe recent australian fire result climate change think 
110116


In [124]:
tweet_2020_sorted = tweet_2020.sort_values(by = 'timestamp')

In [125]:
len(tweet_2020_sorted)

110116

In [126]:
time_cluster = []
for i in range(len(tweet_2020_sorted)):
    time_cluster.append(i // (len(tweet_2020_sorted) / 10) + 1)
    
len(time_cluster)

110116

In [127]:
tweet_2020_sorted['time_cluster'] = time_cluster

In [128]:
tweet_2020_sorted

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,preprop,time_cluster
67045,False,"[auspol, AustraliaOnFire]",[],False,False,0,[https://twitter.com/RDNS_TAI/status/121183481...,,0,[],...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-01T01:15:47,1577841347,1212180587672289281,/AYi51887990/status/1212180587672289281,1176387583011516416,A Yi,,water commidity bought sold highest bidder ele...,1.0
67044,False,[australiaonfire],[],False,True,0,[],1212160929225486342,0,"[{'screen_name': 'padraigegan', 'user_id': '10...",...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-01T02:12:19,1577844739,1212194810888515584,/Azurepink/status/1212194810888515584,74924332,Azure,,great photo photography great ho interest woul...,1.0
67043,False,"[AustraliaOnFire, AusPolhttps]",[],False,False,1,[https://twitter.com/JoeHockey/status/12119845...,,0,[],...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-01T03:49:39,1577850579,1212219308790575104,/CCC_Communities/status/1212219308790575104,854046308,Caring Communities,,people distressed fool,1.0
67042,True,[auspol],[https://pbs.twimg.com/media/ENLpTYeVUAUF9BA.j...,False,False,0,[],,0,[],...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-01T08:21:45,1577866905,1212287785324797952,/lunar_aphrodite/status/1212287785324797952,65484074,Charis Constantine,,true compassion true heartbreak true leadership,1.0
67041,True,"[Wildfires, AustraliaOnFire, FearGod, ObeyGod,...",[https://pbs.twimg.com/media/ENFxsTjX0AA6CTX.jpg],False,False,1,[https://cbsn.ws/37u9sD7],,0,[],...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-01T17:05:02,1577898302,1212419473900216322,/Frukta_Gud/status/1212419473900216322,921809679180881923,Frukta Gud,,lord thy god consuming fire even jealous godde...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,True,"[artists, koala, AustraliaFires, digitalart, i...",[https://pbs.twimg.com/media/ERLItxxW4AYErz6.jpg],False,False,1,[],,0,[],...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-19T23:03:26,1582153406,1230266672235282432,/MobeArt/status/1230266672235282432,1170739797952012294,mobe_art,,redid whole thingare koala best saddens happened,10.0
98999,False,[AustraliaBushfires],[],False,False,1,[],,0,[],...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-19T23:09:59,1582153799,1230268321108578304,/stgregoryscps/status/1230268321108578304,776894294703628288,St Gregory's,,amazing thoughtful mini vinnies organised cake...,10.0
680,False,"[auspol, AustraliaFires, FireFightAustraliahttps]",[],False,False,0,[https://insidestory.org.au/time-to-think-diff...,,0,[],...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-19T23:43:48,1582155828,1230276830055133184,/Jayrgoodall/status/1230276830055133184,3149027809,Jay Goodall,,response call think differently- differently v...,10.0
679,True,"[wildfires, donationsforAustralia, AustraliaFi...",[https://pbs.twimg.com/media/ERLSoyeUEAECQ3f.jpg],False,False,1,[],,0,[],...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-19T23:45:29,1582155929,1230277256179666947,/SarahHa46398397/status/1230277256179666947,1228502896414560256,Sarah Hart,,think international donation going directly ne...,10.0


In [130]:
tweet_2020_1 = tweet_2020_sorted.query('time_cluster == 7')
print(len(tweet_2020_1))

11012


In [131]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_1 = tfidf_vectorizer.fit_transform(np.array(tweet_2020_1['preprop']))

In [132]:
kmeans = KMeans(n_clusters=3).fit(tfidf_1)

In [133]:
cluster_of_3 = []
for i in range(len(tweet_2020_1)):
    cluster_of_3.append(kmeans.predict(tfidf_1.getrow(i))[0])

In [134]:
tweet_2020_1['cluster_3'] = np.array(cluster_of_3)

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [135]:
tweet_2020_1_0 = tweet_2020_1.query('cluster_3 == 0')
tweet_2020_1_1 = tweet_2020_1.query('cluster_3 == 1')
tweet_2020_1_2 = tweet_2020_1.query('cluster_3 == 2')

In [136]:
print(len(tweet_2020_1_0))
print(len(tweet_2020_1_1))
print(len(tweet_2020_1_2))

9019
761
1232


In [137]:
print(len(tfidf_1.todense()))
print(len(tweet_2020_1))

11012
11012


В таблице tweet_2020_1 прилепить каждому тексту его tfidf и разницу с центром кластера.

In [138]:
tweet_2020_1

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,preprop,time_cluster,cluster_3
12008,False,[AustraliaFires],[],False,False,0,[],,0,[],...,2020-01-10T09:38:28,1578649108,1215568580240183297,/chocol8boomboom/status/1215568580240183297,140107883,Noir_Pappillion,,know read heard news loss aboriginee life imag...,7.0,0
12007,True,"[AustraliaFires, KangarooIsland]",[https://pbs.twimg.com/media/EN6RMWrUEAAomHk.j...,False,False,1,[http://bit.ly/2TaizF2],,0,[],...,2020-01-10T09:38:32,1578649112,1215568596585218048,/michaellinder/status/1215568596585218048,20752677,Michael Linder,,"want burn save say wildlife park owner 25,000 ...",7.0,0
12006,False,"[climateprotest, AustraliaFires, VicBushfires,...",[],True,False,9,[https://www.theage.com.au/national/victoria/t...,,2,[],...,2020-01-10T09:38:38,1578649118,1215568621851668480,/FarrahTomazin/status/1215568621851668480,187012604,Farrah Tomazin,,thousand attend climate change rally melbourne...,7.0,1
12005,False,"[realheroes, legends, AustralianBushfireDisast...",[],False,True,1,[],1215194398721368065,0,"[{'screen_name': 'RFSCommissioner', 'user_id':...",...,2020-01-10T09:38:46,1578649126,1215568656509259777,/cammudge/status/1215568656509259777,23683282,cameron mudge,,hero people firefighter,7.0,0
105543,True,"[ThesePeopleAreSick, AustraliaBushfires, RainF...",[https://pbs.twimg.com/media/EN6RV6dX0AUNSid.jpg],True,False,28,[],,5,[],...,2020-01-10T09:39:13,1578649153,1215568769105518592,/jay2kQ17/status/1215568769105518592,392078200,jay2k 🇩🇪🇺🇸,,climate change arse almost always arson cc gre...,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104722,False,[AustraliaBushfires],[],False,False,1,[https://twitter.com/i/events/1216247637533286...,,0,[],...,2020-01-12T19:02:53,1578855773,1216435397758201856,/_bindle/status/1216435397758201856,26046686,PBY,,sweet potato carrot airdropped help endangered...,7.0,2
116167,False,[AustralianBushfireDisaster],[],False,False,0,[https://twitter.com/WordswithSteph/status/121...,,0,[],...,2020-01-12T19:03:20,1578855800,1216435510249435137,/Kempo50/status/1216435510249435137,235718672,KBK📎,,heartbreaking hundred koala perished little ko...,7.0,0
8479,False,"[AustraliaFires, auspol]",[],True,False,31,[https://www.smh.com.au/environment/conservati...,,4,[],...,2020-01-12T19:03:39,1578855819,1216435590192742401,/p_hannam/status/1216435590192742401,495650403,Peter Hannam,,also new ecological tragedy 50m federal cash a...,7.0,0
104721,False,"[live, Fortnite, CharityStream, AustraliaBushf...",[],False,False,2,[],,0,[],...,2020-01-12T19:06:08,1578855968,1216436213571256326,/Rougez_/status/1216436213571256326,1004533605186535424,Rougez⏸,,playing,7.0,0


In [139]:
tfidf_1.todense()[0]

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [140]:
kmeans.cluster_centers_[2]

array([0.        , 0.00359456, 0.        , ..., 0.        , 0.00028284,
       0.        ])

In [141]:
distances = []
for i in range(len(tweet_2020_1)):
    center = np.array(tweet_2020_1['cluster_3'])[i]
    vec_sub = kmeans.cluster_centers_[center] - tfidf_1.todense()[i]
    print(i)
    vec_np = np.array(vec_sub)
    vec_np_2 = vec_np ** 2
    dist = vec_np_2.sum()
    distances.append(dist)
    print(i)

0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
152


1042
1042
1043
1043
1044
1044
1045
1045
1046
1046
1047
1047
1048
1048
1049
1049
1050
1050
1051
1051
1052
1052
1053
1053
1054
1054
1055
1055
1056
1056
1057
1057
1058
1058
1059
1059
1060
1060
1061
1061
1062
1062
1063
1063
1064
1064
1065
1065
1066
1066
1067
1067
1068
1068
1069
1069
1070
1070
1071
1071
1072
1072
1073
1073
1074
1074
1075
1075
1076
1076
1077
1077
1078
1078
1079
1079
1080
1080
1081
1081
1082
1082
1083
1083
1084
1084
1085
1085
1086
1086
1087
1087
1088
1088
1089
1089
1090
1090
1091
1091
1092
1092
1093
1093
1094
1094
1095
1095
1096
1096
1097
1097
1098
1098
1099
1099
1100
1100
1101
1101
1102
1102
1103
1103
1104
1104
1105
1105
1106
1106
1107
1107
1108
1108
1109
1109
1110
1110
1111
1111
1112
1112
1113
1113
1114
1114
1115
1115
1116
1116
1117
1117
1118
1118
1119
1119
1120
1120
1121
1121
1122
1122
1123
1123
1124
1124
1125
1125
1126
1126
1127
1127
1128
1128
1129
1129
1130
1130
1131
1131
1132
1132
1133
1133
1134
1134
1135
1135
1136
1136
1137
1137
1138
1138
1139
1139
1140
1140
1141
1141


1862
1862
1863
1863
1864
1864
1865
1865
1866
1866
1867
1867
1868
1868
1869
1869
1870
1870
1871
1871
1872
1872
1873
1873
1874
1874
1875
1875
1876
1876
1877
1877
1878
1878
1879
1879
1880
1880
1881
1881
1882
1882
1883
1883
1884
1884
1885
1885
1886
1886
1887
1887
1888
1888
1889
1889
1890
1890
1891
1891
1892
1892
1893
1893
1894
1894
1895
1895
1896
1896
1897
1897
1898
1898
1899
1899
1900
1900
1901
1901
1902
1902
1903
1903
1904
1904
1905
1905
1906
1906
1907
1907
1908
1908
1909
1909
1910
1910
1911
1911
1912
1912
1913
1913
1914
1914
1915
1915
1916
1916
1917
1917
1918
1918
1919
1919
1920
1920
1921
1921
1922
1922
1923
1923
1924
1924
1925
1925
1926
1926
1927
1927
1928
1928
1929
1929
1930
1930
1931
1931
1932
1932
1933
1933
1934
1934
1935
1935
1936
1936
1937
1937
1938
1938
1939
1939
1940
1940
1941
1941
1942
1942
1943
1943
1944
1944
1945
1945
1946
1946
1947
1947
1948
1948
1949
1949
1950
1950
1951
1951
1952
1952
1953
1953
1954
1954
1955
1955
1956
1956
1957
1957
1958
1958
1959
1959
1960
1960
1961
1961


2682
2682
2683
2683
2684
2684
2685
2685
2686
2686
2687
2687
2688
2688
2689
2689
2690
2690
2691
2691
2692
2692
2693
2693
2694
2694
2695
2695
2696
2696
2697
2697
2698
2698
2699
2699
2700
2700
2701
2701
2702
2702
2703
2703
2704
2704
2705
2705
2706
2706
2707
2707
2708
2708
2709
2709
2710
2710
2711
2711
2712
2712
2713
2713
2714
2714
2715
2715
2716
2716
2717
2717
2718
2718
2719
2719
2720
2720
2721
2721
2722
2722
2723
2723
2724
2724
2725
2725
2726
2726
2727
2727
2728
2728
2729
2729
2730
2730
2731
2731
2732
2732
2733
2733
2734
2734
2735
2735
2736
2736
2737
2737
2738
2738
2739
2739
2740
2740
2741
2741
2742
2742
2743
2743
2744
2744
2745
2745
2746
2746
2747
2747
2748
2748
2749
2749
2750
2750
2751
2751
2752
2752
2753
2753
2754
2754
2755
2755
2756
2756
2757
2757
2758
2758
2759
2759
2760
2760
2761
2761
2762
2762
2763
2763
2764
2764
2765
2765
2766
2766
2767
2767
2768
2768
2769
2769
2770
2770
2771
2771
2772
2772
2773
2773
2774
2774
2775
2775
2776
2776
2777
2777
2778
2778
2779
2779
2780
2780
2781
2781


3502
3502
3503
3503
3504
3504
3505
3505
3506
3506
3507
3507
3508
3508
3509
3509
3510
3510
3511
3511
3512
3512
3513
3513
3514
3514
3515
3515
3516
3516
3517
3517
3518
3518
3519
3519
3520
3520
3521
3521
3522
3522
3523
3523
3524
3524
3525
3525
3526
3526
3527
3527
3528
3528
3529
3529
3530
3530
3531
3531
3532
3532
3533
3533
3534
3534
3535
3535
3536
3536
3537
3537
3538
3538
3539
3539
3540
3540
3541
3541
3542
3542
3543
3543
3544
3544
3545
3545
3546
3546
3547
3547
3548
3548
3549
3549
3550
3550
3551
3551
3552
3552
3553
3553
3554
3554
3555
3555
3556
3556
3557
3557
3558
3558
3559
3559
3560
3560
3561
3561
3562
3562
3563
3563
3564
3564
3565
3565
3566
3566
3567
3567
3568
3568
3569
3569
3570
3570
3571
3571
3572
3572
3573
3573
3574
3574
3575
3575
3576
3576
3577
3577
3578
3578
3579
3579
3580
3580
3581
3581
3582
3582
3583
3583
3584
3584
3585
3585
3586
3586
3587
3587
3588
3588
3589
3589
3590
3590
3591
3591
3592
3592
3593
3593
3594
3594
3595
3595
3596
3596
3597
3597
3598
3598
3599
3599
3600
3600
3601
3601


...

7602
7602
7603
7603
7604
7604
7605
7605
7606
7606
7607
7607
7608
7608
7609
7609
7610
7610
7611
7611
7612
7612
7613
7613
7614
7614
7615
7615
7616
7616
7617
7617
7618
7618
7619
7619
7620
7620
7621
7621
7622
7622
7623
7623
7624
7624
7625
7625
7626
7626
7627
7627
7628
7628
7629
7629
7630
7630
7631
7631
7632
7632
7633
7633
7634
7634
7635
7635
7636
7636
7637
7637
7638
7638
7639
7639
7640
7640
7641
7641
7642
7642
7643
7643
7644
7644
7645
7645
7646
7646
7647
7647
7648
7648
7649
7649
7650
7650
7651
7651
7652
7652
7653
7653
7654
7654
7655
7655
7656
7656
7657
7657
7658
7658
7659
7659
7660
7660
7661
7661
7662
7662
7663
7663
7664
7664
7665
7665
7666
7666
7667
7667
7668
7668
7669
7669
7670
7670
7671
7671
7672
7672
7673
7673
7674
7674
7675
7675
7676
7676
7677
7677
7678
7678
7679
7679
7680
7680
7681
7681
7682
7682
7683
7683
7684
7684
7685
7685
7686
7686
7687
7687
7688
7688
7689
7689
7690
7690
7691
7691
7692
7692
7693
7693
7694
7694
7695
7695
7696
7696
7697
7697
7698
7698
7699
7699
7700
7700
7701
7701


8422
8422
8423
8423
8424
8424
8425
8425
8426
8426
8427
8427
8428
8428
8429
8429
8430
8430
8431
8431
8432
8432
8433
8433
8434
8434
8435
8435
8436
8436
8437
8437
8438
8438
8439
8439
8440
8440
8441
8441
8442
8442
8443
8443
8444
8444
8445
8445
8446
8446
8447
8447
8448
8448
8449
8449
8450
8450
8451
8451
8452
8452
8453
8453
8454
8454
8455
8455
8456
8456
8457
8457
8458
8458
8459
8459
8460
8460
8461
8461
8462
8462
8463
8463
8464
8464
8465
8465
8466
8466
8467
8467
8468
8468
8469
8469
8470
8470
8471
8471
8472
8472
8473
8473
8474
8474
8475
8475
8476
8476
8477
8477
8478
8478
8479
8479
8480
8480
8481
8481
8482
8482
8483
8483
8484
8484
8485
8485
8486
8486
8487
8487
8488
8488
8489
8489
8490
8490
8491
8491
8492
8492
8493
8493
8494
8494
8495
8495
8496
8496
8497
8497
8498
8498
8499
8499
8500
8500
8501
8501
8502
8502
8503
8503
8504
8504
8505
8505
8506
8506
8507
8507
8508
8508
8509
8509
8510
8510
8511
8511
8512
8512
8513
8513
8514
8514
8515
8515
8516
8516
8517
8517
8518
8518
8519
8519
8520
8520
8521
8521


9242
9242
9243
9243
9244
9244
9245
9245
9246
9246
9247
9247
9248
9248
9249
9249
9250
9250
9251
9251
9252
9252
9253
9253
9254
9254
9255
9255
9256
9256
9257
9257
9258
9258
9259
9259
9260
9260
9261
9261
9262
9262
9263
9263
9264
9264
9265
9265
9266
9266
9267
9267
9268
9268
9269
9269
9270
9270
9271
9271
9272
9272
9273
9273
9274
9274
9275
9275
9276
9276
9277
9277
9278
9278
9279
9279
9280
9280
9281
9281
9282
9282
9283
9283
9284
9284
9285
9285
9286
9286
9287
9287
9288
9288
9289
9289
9290
9290
9291
9291
9292
9292
9293
9293
9294
9294
9295
9295
9296
9296
9297
9297
9298
9298
9299
9299
9300
9300
9301
9301
9302
9302
9303
9303
9304
9304
9305
9305
9306
9306
9307
9307
9308
9308
9309
9309
9310
9310
9311
9311
9312
9312
9313
9313
9314
9314
9315
9315
9316
9316
9317
9317
9318
9318
9319
9319
9320
9320
9321
9321
9322
9322
9323
9323
9324
9324
9325
9325
9326
9326
9327
9327
9328
9328
9329
9329
9330
9330
9331
9331
9332
9332
9333
9333
9334
9334
9335
9335
9336
9336
9337
9337
9338
9338
9339
9339
9340
9340
9341
9341


10052
10052
10053
10053
10054
10054
10055
10055
10056
10056
10057
10057
10058
10058
10059
10059
10060
10060
10061
10061
10062
10062
10063
10063
10064
10064
10065
10065
10066
10066
10067
10067
10068
10068
10069
10069
10070
10070
10071
10071
10072
10072
10073
10073
10074
10074
10075
10075
10076
10076
10077
10077
10078
10078
10079
10079
10080
10080
10081
10081
10082
10082
10083
10083
10084
10084
10085
10085
10086
10086
10087
10087
10088
10088
10089
10089
10090
10090
10091
10091
10092
10092
10093
10093
10094
10094
10095
10095
10096
10096
10097
10097
10098
10098
10099
10099
10100
10100
10101
10101
10102
10102
10103
10103
10104
10104
10105
10105
10106
10106
10107
10107
10108
10108
10109
10109
10110
10110
10111
10111
10112
10112
10113
10113
10114
10114
10115
10115
10116
10116
10117
10117
10118
10118
10119
10119
10120
10120
10121
10121
10122
10122
10123
10123
10124
10124
10125
10125
10126
10126
10127
10127
10128
10128
10129
10129
10130
10130
10131
10131
10132
10132
10133
10133
10134
10134
1013

10736
10736
10737
10737
10738
10738
10739
10739
10740
10740
10741
10741
10742
10742
10743
10743
10744
10744
10745
10745
10746
10746
10747
10747
10748
10748
10749
10749
10750
10750
10751
10751
10752
10752
10753
10753
10754
10754
10755
10755
10756
10756
10757
10757
10758
10758
10759
10759
10760
10760
10761
10761
10762
10762
10763
10763
10764
10764
10765
10765
10766
10766
10767
10767
10768
10768
10769
10769
10770
10770
10771
10771
10772
10772
10773
10773
10774
10774
10775
10775
10776
10776
10777
10777
10778
10778
10779
10779
10780
10780
10781
10781
10782
10782
10783
10783
10784
10784
10785
10785
10786
10786
10787
10787
10788
10788
10789
10789
10790
10790
10791
10791
10792
10792
10793
10793
10794
10794
10795
10795
10796
10796
10797
10797
10798
10798
10799
10799
10800
10800
10801
10801
10802
10802
10803
10803
10804
10804
10805
10805
10806
10806
10807
10807
10808
10808
10809
10809
10810
10810
10811
10811
10812
10812
10813
10813
10814
10814
10815
10815
10816
10816
10817
10817
10818
10818
1081

In [142]:
tweet_2020_1['distances'] = distances

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [143]:
type(tweet_2020_1)

pandas.core.frame.DataFrame

In [168]:
length = []
for i in range(len(tweet_2020_1['preprop'])):
    ll = len(np.array(tweet_2020_1['preprop'])[i].split())
    length.append(ll)

In [171]:
tweet_2020_1['preprop_len'] = length

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [173]:
tweet_2020_1.sort_values(by = 'distances').query('cluster_3 == 0 and preprop != "" and preprop_len > 3 ')

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,tweet_url,user_id,username,video_url,preprop,time_cluster,cluster_3,distances,preprop len,preprop_len
11995,False,"[Australia, AustralianWildFires, AustraliaFires]",[],False,False,0,[https://twitter.com/figstyles/status/12155317...,,0,[],...,/fersan_bob/status/1215569292839849984,1215372672596619264,Bob Chun,,thank australian people nwildfires fire,7.0,0,0.965142,5,5
94597,False,"[AustraliaFire, AustralianBushfire]",[],False,False,0,[https://youtu.be/oc0fKMBfGbY],,0,[],...,/reduhimel/status/1215756703284654080,899557488,Redowan Ullah Himel,,fire animal burning video footage fire news,7.0,0,0.966194,7,7
119189,True,"[AustraliaFires, AustralianBushfireDisaster]",[https://pbs.twimg.com/media/EN7SK7CW4AcWLMo.jpg],True,False,135,[],,5,[],...,/m_abdalla47/status/1215640037699833856,855799023357349889,Muse Kamara,,beautiful animal saved australian fire,7.0,0,0.968107,5,5
11275,True,"[AustraliaFires, AustralianBushfireDisasterpic]",[https://pbs.twimg.com/media/EN7SK7CW4AcWLMo.jpg],True,False,135,[],,5,[],...,/m_abdalla47/status/1215640037699833856,855799023357349889,Muse Kamara,,beautiful animal saved australian fire,7.0,0,0.968107,5,5
44610,False,"[AustralianBushfireDisaster, AustraliaOnFire, ...",[],True,False,3,[https://twitter.com/i/events/1215655756814594...,,2,[],...,/Nucha73637221/status/1215708434248552448,1020875118787026945,Lins 🇭🇲,,disturbing news break heart fire beautiful cou...,7.0,0,0.968213,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75999,False,[AustraliaBurning],[],False,False,0,[https://twitter.com/cnni/status/1216347501793...,,0,[],...,/shooter384/status/1216350986677170182,1018500574931759105,Daniele,,thor asgard odin son,7.0,0,1.005529,4,4
117855,True,[AustralianBushfireDisaster],[https://pbs.twimg.com/media/EN9tRQCXsAUr0B9.jpg],False,False,0,[],,0,[],...,/Su67338460/status/1215810602607706113,1207460639788191744,Su,,dios tiene la ultima palabra,7.0,0,1.005540,5,5
118706,False,"[NSWfires, AustralianBushfireDisaster]",[],False,False,1,[],,0,[],...,/WizardofWoyWoy/status/1215729938243829760,949210736613445632,Wizard of Woy Woy,,blowering dam pronounced belowring,7.0,0,1.005615,4,4
119272,False,"[lg, mobile, smartphone, PROFIT, profits, Aust...",[],False,False,0,[https://www.whateverison.com/lg-has-confidenc...,,0,[],...,/Twistyapps2/status/1215627926726725639,1132899003807485952,Twistyapps,,lg confidence smartphone profitable,7.0,0,1.005674,4,4


In [178]:
list(tweet_2020_1.sort_values(by = 'distances').query('cluster_3 == 0 and preprop != "" and preprop_len > 3 ')['text'])

['Thank You Australian People !  nWildFires Fires …',
 'Australia fire animals are burning video footage fire news  ',
 'One of the most beautiful animals that was saved from the Australian fires  ',
 'One of the most beautiful animals that was saved from the Australian fires ',
 'Disturbing news...breaks my heart what these fires are doing to this beautiful country      “1 billion animals feared dead in Australia’s fires. It may get worse” by …',
 'A billion animals have been caught in Australia’s fires. Some may go extinct. …   Fires ',
 'Support animal rescue efforts by making a gift to Animal Rescue Fund (via   Fires OnFire nBushfire Burning',
 'The Australian Fires are making me love koalas now and I hope that a lot of them get saved!     ',
 '“I want you to act like your house is on fire, because it is.” Fires BushFires    ',
 'In Australia The Things Are Getting Worse.. As The Fire Causing Dead To More That 100 Crores Animals ! So Donate Any Amount That You Can !  Bushfires OnFi

In [181]:
def summarization(n):
    tweet_2020_n = tweet_2020_sorted.query('time_cluster == %d' % n)
    tfidf_n = tfidf_vectorizer.fit_transform(np.array(tweet_2020_n['preprop']))
    kmeans = KMeans(n_clusters=3).fit(tfidf_n)

    print("End Kmeans")
    cluster_of_3 = []
    for i in range(len(tweet_2020_n)):
        cluster_of_3.append(kmeans.predict(tfidf_n.getrow(i))[0])

    tweet_2020_n['cluster_3'] = np.array(cluster_of_3)

    distances = []
    for i in range(len(tweet_2020_n)):
        center = np.array(tweet_2020_n['cluster_3'])[i]
        vec_sub = kmeans.cluster_centers_[center] - tfidf_n.todense()[i]
        if i % 50 == 0:
            print(i)
        vec_np = np.array(vec_sub)
        vec_np_2 = vec_np ** 2
        dist = vec_np_2.sum()
        distances.append(dist)
    
    tweet_2020_n['distances'] = distances
    print("End distances")
    
    length = []
    for i in range(len(tweet_2020_n['preprop'])):
        ll = len(np.array(tweet_2020_n['preprop'])[i].split())
        length.append(ll)

    tweet_2020_n['preprop_len'] = length

    return tweet_2020_n

In [182]:
tweet_2020_5 = summarization(5)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [183]:
list(tweet_2020_5.sort_values(by = 'distances').query('cluster_3 == 0 and preprop != "" and preprop_len > 3 ')['text'])

['      The world has 195 Countries ... only ONE Country on fire? This is not Climate Change, this is a ',
 "Australian Fires. Evidence of DEW'S in use!… nWildFires Fires Fire BushFire BushFires     ",
 '      There goes your “Climate Is The Cause” for the fires in Australia   …',
 'Climate Change? Nearly 200 People Arrested Across Australia For Deliberately Starting Bushfires …  nBushfiresDisaster Fires ',
 'The world : Australian people is facing the biggest fire ever. Forests are being destroyed. Climate change. People : The world : Also Koalas are dyingPeople : ',
 'So answer this for me: The fires in Australia are due to climate change, yet they just arrested 24 people for deliberately setting these fires?   ',
 'The fires are still not caused be climate change. …    ',
 'Sending prayers to all affected. Attached is a list of organizations of Australian local state services helping put the fires out. Let’s show them support in their time of need.   fires …',
 'Our hearts break for

In [186]:
tweet_2020_5.to_csv('tweet_2020_5.csv')

In [184]:
tweet_2020_6 = summarization(6)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [185]:
list(tweet_2020_6.sort_values(by = 'distances').query('cluster_3 == 0 and preprop != "" and preprop_len > 3 ')['text'])

['Well over 1 billion animals killed by fires.  People trapped on the water.  The future of climate change is here but they still want to dig up more coal.  …',
 ' to ? Here’s what you need to know  via   Fires ',
 "The devastating  have left at least 25 people dead, 20 million acres burned, and a billion native animals killed. For many Australians, it's past time for their government to get serious about the climate crisis. ",
 'Australian Bush Fires - Not Climate Change! They Were Started By Arsonists -    ',
 'Since i want to go to OZ one day. For months now I have been reading a lot of sad and heartbreaking news that bushfires has been going on in the country Australia.   Fires Climate change is the factor of it. ',
 'My heart goes out to all the people and animals who are in Australia fires right now. I wish I had money right now to donate for you guys. Im sorry I can only send love and prayers.  ',
 '   Hope the aboriginal people who live in the places where some of these fires a

In [187]:
tweet_2020_6.to_csv('tweet_2020_6.csv')

In [188]:
tweet_2020_4 = summarization(4)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [189]:
tweet_2020_4.to_csv('tweet_2020_4.csv')

In [190]:
tweet_2020_3 = summarization(3)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [191]:
tweet_2020_3.to_csv('tweet_2020_3.csv')

In [192]:
tweet_2020_2 = summarization(2)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [193]:
tweet_2020_1 = summarization(1)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [194]:
tweet_2020_7 = summarization(7)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [195]:
tweet_2020_8 = summarization(8)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [196]:
tweet_2020_9 = summarization(9)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [197]:
tweet_2020_10 = summarization(10)

End Kmeans


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


End distances


/home/nicklake00/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [198]:
tweet_2020_2.to_csv('tweet_2020_2.csv')
tweet_2020_1.to_csv('tweet_2020_1.csv')
tweet_2020_7.to_csv('tweet_2020_7.csv')
tweet_2020_8.to_csv('tweet_2020_8.csv')
tweet_2020_9.to_csv('tweet_2020_9.csv')
tweet_2020_10.to_csv('tweet_2020_10.csv')

In [203]:
from tqdm import tqdm

for i in tqdm(range(99900000)):
    pass
    #print(i**2)